# Laboratorio opcional: Retropropagación mediante un gráfico de cálculo
Este laboratorio le permitirá comprender mejor un algoritmo clave utilizado por la mayoría de los marcos de aprendizaje automático. El descenso gradiente requiere la derivada del coste con respecto a cada parámetro de la red.  Las redes neuronales pueden tener millones o incluso miles de millones de parámetros. El algoritmo de *propagación hacia atrás* se utiliza para calcular esas derivadas. Los *gráficos de cálculo* se utilizan para simplificar la operación. Profundicemos en esto a continuación.


In [1]:
from sympy import *
import numpy as np
import re
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.widgets import TextBox
from matplotlib.widgets import Button
import ipywidgets as widgets
from lab_utils_backprop import *

## Computation Graph
Un gráfico de cálculo simplifica el cálculo de derivadas complejas dividiéndolo en pasos más pequeños. Veamos cómo funciona.

Calculemos la derivada de esta expresión ligeramente compleja, $J = (2+3w)^2$. Queremos hallar la derivada de $J$ con respecto a $w$ o $\frac{\partial J}{\partial w}$.

In [2]:
plt.close("all")
plt_network(config_nw0, "./images/C2_W2_BP_network0.PNG")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Arriba puedes ver que hemos dividido la expresión en dos nodos en los que podemos trabajar de forma independiente. Si ya tienes una buena comprensión del proceso de la clase, puedes seguir adelante y rellenar las casillas en el diagrama anterior. Primero debes rellenar las casillas azules que van de izquierda a derecha y luego rellenar las casillas verdes empezando por la derecha y moviéndote hacia la izquierda.
Si tienes los valores correctos, los valores se mostrarán como verde o azul. Si el valor es incorrecto, aparecerá en rojo. Ten en cuenta que el gráfico interactivo no es especialmente robusto. Si tiene problemas con la interfaz, ejecute de nuevo la celda anterior para reiniciar.

Si no está seguro del proceso, trabajaremos este ejemplo paso a paso a continuación.

### Propagación hacia delante   
Vamos a calcular los valores en la dirección hacia adelante.

>Sólo una nota sobre esta sección. Utiliza variables globales y las reutiliza a medida que avanza el cálculo. Si ejecutas celdas fuera de orden, puedes obtener resultados raros. Si lo haces, vuelve a este punto y ejecútalas en orden.

In [3]:
w = 3
a = 2+3*w
J = a**2
print(f"a = {a}, J = {J}")

a = 11, J = 121


Puede rellenar estos valores en las casillas azules de arriba.

### Backprop
<img align="left" src="./images/C2_W2_BP_network0_j.PNG"     style=" width:100px; padding: 10px 20px; " > Backprop es el algoritmo que utilizamos para calcular las derivadas. Como se describe en las conferencias, backprop comienza a la derecha y se mueve hacia la izquierda. El primer nodo a considerar es $J = a^2 $ y el primer paso es encontrar $\frac{\partial J}{\partial a}$ 


### $\frac{\partial J}{\partial a}$ 
#### Arithmetically
Hallar $\frac{\partial J}{\partial a}$ hallando cómo cambia $J$ como resultado de un pequeño cambio en $a$. Esto se describe en detalle en el laboratorio opcional de derivadas.

In [4]:
a_epsilon = a + 0.001       # a epsilon
J_epsilon = a_epsilon**2    # J_epsilon
k = (J_epsilon - J)/0.001   # difference divided by epsilon
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_da ~= k = {k} ")

J = 121, J_epsilon = 121.02200099999999, dJ_da ~= k = 22.000999999988835 


$\frac{\partial J}{\partial a}$ es 22 que es $2\times a$. Nuestro resultado no es exactamente 2 veces a$ porque nuestro valor épsilon no es infinitesimalmente pequeño. 
#### Simbólicamente
Ahora, vamos a utilizar SymPy para calcular derivadas simbólicamente como lo hicimos en el laboratorio opcional de derivadas. Antepondremos al nombre de la variable una 's' para indicar que se trata de una variable *simbólica*.

In [5]:
sw,sJ,sa = symbols('w,J,a')
sJ = sa**2
sJ

a**2

In [6]:
sJ.subs([(sa,a)])

121

In [7]:
dJ_da = diff(sJ, sa)
dJ_da

2*a

Entonces, $\frac{\partial J}{\partial a} = 2a$. Cuando $a=11$, $\frac{\partial J}{\partial a} = 22$. Esto coincide con nuestro cálculo aritmético anterior.
Si aún no lo has hecho, puedes volver al diagrama anterior y rellenar el valor de $\frac{\partial J}{parcial a}$.

### $\frac{\partial J}{\partial w}$ 
<img align="left" src="./images/C2_W2_BP_network0_a.PNG"     style=" width:100px; padding: 10px 20px; " >  Moviéndonos de derecha a izquierda, el siguiente valor que nos gustaría calcular es $\frac{\partial J}{\partial w}$. Para ello, primero tenemos que calcular $\frac{parcial a}{parcial w}$ que describe cómo la salida de este nodo, $a$, cambia cuando la entrada $w$ cambia un poco.

#### Aritméticamente
Hallar $\frac{\partial a}{\partial w}$ hallando cómo cambia $a$ como resultado de un pequeño cambio en $w$.

In [8]:
w_epsilon = w + 0.001       # a  plus a small value, epsilon
a_epsilon = 2 + 3*w_epsilon
k = (a_epsilon - a)/0.001   # difference divided by epsilon
print(f"a = {a}, a_epsilon = {a_epsilon}, da_dw ~= k = {k} ")

a = 11, a_epsilon = 11.003, da_dw ~= k = 3.0000000000001137 


Calculado aritméticamente, $\frac{\partial a}{\partial w} \approx 3$. Vamos a probarlo con SymPy.

In [9]:
sa = 2 + 3*sw
sa

3*w + 2

In [10]:
da_dw = diff(sa,sw)
da_dw

3

>El siguiente paso es la parte interesante:
> - Sabemos que un pequeño cambio en $w$ hará que $a$ cambie 3 veces esa cantidad.
> - Sabemos que un pequeño cambio en $a$ hará que $J$ cambie 2 veces $a$ esa cantidad. (a=11 en este ejemplo)    
 Entonces, juntando todo esto,  
> - Sabemos que un pequeño cambio en $w$ hará que $J$ cambie $3 veces 2 veces a$ esa cantidad.
> 
> Estos cambios en cascada se conocen como *la regla de la cadena*.  Se puede escribir así: 
 $$\frac{\partial J}{\partial w} = \frac{\partial a}{\partial w} \frac {parcial J} {parcial a} $$
 
Merece la pena dedicar algo de tiempo a pensar en esto si no está claro. Este es un punto clave.
 
 Intentemos calcularlo:
 

In [11]:
dJ_dw = da_dw * dJ_da
dJ_dw

6*a

Y $a$ es 11 en este ejemplo por lo que $\frac{\partial J}{\partial w} = 66$. Podemos comprobar esto aritméticamente:

In [12]:
w_epsilon = w + 0.001
a_epsilon = 2 + 3*w_epsilon
J_epsilon = a_epsilon**2
k = (J_epsilon - J)/0.001   # difference divided by epsilon
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_dw ~= k = {k} ")

J = 121, J_epsilon = 121.06600900000001, dJ_dw ~= k = 66.0090000000082 


¡OK! Ahora puede rellenar los valores de $\frac{\partial a}{\partial w}$ y $\frac{\partial J}{\partial w}$ en el diagrama si aún no lo ha hecho. 

**Otro punto de vista**  
Uno podría visualizar estos cambios en cascada de esta manera:  
<img align="center" src="./images/C2_W2_BP_network0_diff.PNG" style=" width:500px; padding: 10px 20px; " >  
Un pequeño cambio en $w$ se multiplica por $\frac{\partial a}{\partial w}$ resultando en un cambio que es 3 veces más grande. Este cambio más grande se multiplica entonces por $\frac{\partial J}{\partial a}$ resultando en un cambio que ahora es $3 \times 22 = 66$ veces más grande.

## Gráfico de cálculo de una red neuronal simple
A continuación se muestra un gráfico de la red neuronal utilizada en la clase con diferentes valores. Intenta rellenar los valores de las casillas. Tenga en cuenta que el gráfico interactivo no es especialmente robusto. Si tiene problemas con la interfaz, ejecute de nuevo la celda de abajo para reiniciar.

In [13]:
plt.close("all")
plt_network(config_nw1, "./images/C2_W2_BP_network1.PNG")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

A continuación, repasaremos en detalle los cálculos necesarios para rellenar el gráfico de cálculo anterior. Empezaremos por la ruta de avance.

### Propagación hacia delante
Los cálculos de la propagación hacia delante son los que has aprendido recientemente para las redes neuronales. Puedes comparar los valores siguientes con los que calculaste para el diagrama anterior.

In [14]:
# Inputs and parameters
x = 2
w = -2
b = 8
y = 1
# calculate per step values   
c = w * x
a = c + b
d = a - y
J = d**2/2
print(f"J={J}, d={d}, a={a}, c={c}")

J=4.5, d=3, a=4, c=-4


### Backward propagation (Backprop)
<img align="left" src="./images/C2_W2_BP_network1_jdsq.PNG"     style=" width:100px; padding: 10px 20px; " > Como se describe en las conferencias, backprop comienza a la derecha y se mueve hacia la izquierda. El primer nodo a considerar es $J = \frac{1}{2}d^2 $ y el primer paso es encontrar $\frac{\partial J}{\partial d}$. 


### $\frac{\partial J}{\partial d}$ 

#### Aritméticamente
Hallar $\frac{\partial J}{\partial d}$ hallando cómo cambia $J$ como resultado de un pequeño cambio en $d$.

In [15]:
d_epsilon = d + 0.001
J_epsilon = d_epsilon**2/2
k = (J_epsilon - J)/0.001   # difference divided by epsilon
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_dd ~= k = {k} ")

J = 4.5, J_epsilon = 4.5030005, dJ_dd ~= k = 3.0004999999997395 


$\frac{\partial J}{\partial d}$ es 3, que es el valor de $d$. Nuestro resultado no es exactamente $d$ porque nuestro valor épsilon no es infinitesimalmente pequeño. 
#### Simbólicamente
Ahora, vamos a utilizar SymPy para calcular derivadas simbólicamente, como lo hicimos en el laboratorio opcional de derivadas. Antepondremos al nombre de la variable una 's' para indicar que se trata de una variable *simbólica*.

In [16]:
sx,sw,sb,sy,sJ = symbols('x,w,b,y,J')
sa, sc, sd = symbols('a,c,d')
sJ = sd**2/2
sJ

d**2/2

In [17]:
sJ.subs([(sd,d)])

9/2

In [18]:
dJ_dd = diff(sJ, sd)
dJ_dd

d

So, $\frac{\partial J}{\partial d}$ = d. When $d=3$, $\frac{\partial J}{\partial d}$ = 3. This matches our arithmetic calculation above.
If you have not already done so, you can go back to the diagram above and fill in the value for $\frac{\partial J}{\partial d}$.

### $\frac{\partial J}{\partial a}$ 
<img align="left" src="./images/C2_W2_BP_network1_d.PNG"     style=" width:100px; padding: 10px 20px; " >  Moviéndonos de derecha a izquierda, el siguiente valor que nos gustaría calcular es $\frac{\partial J}{\partial a}$. Para ello, primero tenemos que calcular $\frac{\partial d}{\partial a}$ que describe cómo la salida de este nodo cambia cuando la entrada $a$ cambia un poco. (Tenga en cuenta, no estamos interesados en cómo cambia la salida cuando $y $ cambia ya que $y $ no es un parámetro).

#### Aritméticamente
Hallar $\frac{\partial d}{\partial a}$ hallando cómo cambia $d$ como resultado de un pequeño cambio en $a$.

In [19]:
a_epsilon = a + 0.001         # a  plus a small value
d_epsilon = a_epsilon - y
k = (d_epsilon - d)/0.001   # difference divided by epsilon
print(f"d = {d}, d_epsilon = {d_epsilon}, dd_da ~= k = {k} ")

d = 3, d_epsilon = 3.0010000000000003, dd_da ~= k = 1.000000000000334 


Calculado aritméticamente, $\frac{\partial d}{\partial a} \approx 1$. Vamos a probarlo con SymPy.
#### Simbólicamente

In [20]:
sd = sa - sy
sd

a - y

In [21]:
dd_da = diff(sd,sa)
dd_da

1

Calculado aritméticamente, $\frac{\partial d}{\partial a}$ también es igual a 1.  
> El siguiente paso es la parte interesante, que se repite de nuevo en este ejemplo:
> - Sabemos que un pequeño cambio en $a$ hará que $d$ cambie en 1 veces esa cantidad.
> - Sabemos que un pequeño cambio en $d$ hará que $J$ cambie en $d$ veces esa cantidad. (d=3 en este ejemplo)    
 Entonces, juntando todo esto,   
> - Sabemos que un pequeño cambio en $a$ hará que $J$ cambie en $1\ veces d$ esa cantidad.
> 
> Esto es de nuevo *la regla de la cadena*.  Se puede escribir así: 
 $$\frac{\partial J}{\partial a} = \frac{\partial d}{\partial a} \frac{\partial J}{\partial d} $$
 
 Intentemos calcularlo:
 

In [22]:
dJ_da = dd_da * dJ_dd
dJ_da

d

And $d$ is 3 in this example so $\frac{\partial J}{\partial a} = 3$. We can check this arithmetically:

In [23]:
a_epsilon = a + 0.001
d_epsilon = a_epsilon - y
J_epsilon = d_epsilon**2/2
k = (J_epsilon - J)/0.001   
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_da ~= k = {k} ")

J = 4.5, J_epsilon = 4.503000500000001, dJ_da ~= k = 3.0005000000006277 


OK, ¡coinciden! Ahora puede rellenar los valores de $\frac{\partial d}{\partial a}$ y $\frac{\partial J}{\partial a}$ en el diagrama si aún no lo ha hecho. 

> **Los pasos en backprop**   
> Ahora que has trabajado a través de varios nodos, podemos escribir el método básico:\
> trabajando de derecha a izquierda, para cada nodo:
>- calcula la(s) derivada(s) local(es) del nodo.
>- utilizando la regla de la cadena, combínela con la derivada del coste con respecto al nodo de la derecha.   

La(s) derivada(s) local(es) es(son) la(s) derivada(s) de la salida del nodo actual con respecto a todas las entradas o parámetros.

Continuemos con el trabajo. Seremos un poco menos verboso ahora que usted está familiarizado con el método.

### $\frac{\partial J}{\partial c}$,  $\frac{\partial J}{\partial b}$
<img align="left" src="./images/C2_W2_BP_network1_a.PNG"     style=" width:100px; padding: 10px 20px; " >
El siguiente nodo tiene dos derivadas de interés. Tenemos que calcular $\frac{\partial J}{\partial c}$ para que podamos propagar a la izquierda. También queremos calcular $ \frac {\partial J} {\partial b}$. Encontrar la derivada del costo con respecto a los parámetros $w $ y $b $ es el objeto de backprop. 

In [24]:
# calculate the local derivatives da_dc, da_db
sa = sc + sb
sa

b + c

In [25]:
da_dc = diff(sa,sc)
da_db = diff(sa,sb)
print(da_dc, da_db)

1 1


In [26]:
dJ_dc = da_dc * dJ_da
dJ_db = da_db * dJ_da
print(f"dJ_dc = {dJ_dc},  dJ_db = {dJ_db}")

dJ_dc = d,  dJ_db = d


And in our example, d = 3

###  $\frac{\partial J}{\partial w}$
<img align="left" src="./images/C2_W2_BP_network1_c.PNG"     style=" width:100px; padding: 10px 20px; " > Tl último nodo de este ejemplo calcula `c`. Aquí, estamos interesados en cómo cambia J con respecto al parámetro w. No vamos a retropropagar a la entrada $x$, por lo que no estamos interesados en $\frac{\partial J}{\partial x}$. Vamos a empezar por calcular $ \frac {\partial c} {\partial w} $.

In [27]:
# calculate the local derivative
sc = sw * sx
sc

w*x

In [28]:
dc_dw = diff(sc,sw)
dc_dw

x

Esta derivada es un poco más emocionante que la anterior. Esto variará dependiendo del valor de $x $. En nuestro ejemplo es 2.

Combine esto con $\frac{\partial J}{\partial c}$ to find $\frac{\partial J}{\partial w}$.

In [32]:
dJ_dw = dc_dw * dJ_dc
dJ_dw

d*x

In [33]:
print(f"dJ_dw = {dJ_dw.subs([(sd,d),(sx,x)])}")

dJ_dw = 2*d


$d=3$, por lo que $\frac{\partial J}{\partial w} = 6$ para nuestro ejemplo.   
Vamos a probar esto aritméticamente:

In [34]:
J_epsilon = ((w+0.001)*x+b - y)**2/2
k = (J_epsilon - J)/0.001  
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_dw ~= k = {k} ")

J = 4.5, J_epsilon = 4.506002, dJ_dw ~= k = 6.001999999999619 


¡Coinciden! Estupendo. Puedes añadir $\frac{\partial J}{\partial w}$ al diagrama anterior y nuestro análisis está completo.

## ¡Enhorabuena!
Has trabajado a través de un ejemplo de propagación hacia atrás utilizando un grafo de computación. Puedes aplicar esto a ejemplos más grandes siguiendo el mismo enfoque nodo por nodo.